# Arduino-ROS Ultrasonic Distance Sensor

One of the useful sensors in robots are the range sensors. One of the cheapest range sensor is the ultrasonic distance sensor. The ultrasonic sensor has two pins for handling input and output, called Echo and Trigger. We are using the HC-SR04 ultrasonic distance sensor and the circuit is shown in the following image:


<img src="img/img3.png" alt="Drawing" style="width: 400px;"/>

The ultrasonic sound sensor contains two sections: one is the transmitter and the other is the receiver. The working of the ultrasonic distance sensor is, when a trigger pulse of a short duration is applied to the trigger pin of the ultrasonic sensors, the ultrasonic transmitter sends the sound signals to the robot environment. The sound signal sent from the transmitter hits on some obstacles and is reflected back to the sensor. The reflected sound waves are collected by the ultrasonic receiver, generating an output signal which has a relation to the time required to receive the reflected sound signals.

## Equations to  find distance using the ultrasonic range sensor

Following are the equations used to compute the distance from an ultrasonic range sensor to an obstacle:

**Distance = Speed * Time/2**

**Speed of sound at sea level = 343 m/s or 34300 cm/s**

**Thus, Distance = 17150 * Time (unit cm)**


We can compute the distance to the obstacle using the pulse duration of the output. Following is the code to work with the ultrasonic sound sensor and send value through the ultrasound topic using the range message definition in ROS:


In [ ]:
#include <ros.h>
#include <ros/time.h>
#include <sensor_msgs/Range.h>
   
#define echoPin 7 // Echo Pin
#define trigPin 8 // Trigger Pin
   
int maximumRange = 200; // Maximum range needed
int minimumRange = 0; // Minimum range needed
long duration, distance; // Duration used to calculate distance

ros::NodeHandle  nh;
sensor_msgs::Range range_msg;
   
ros::Publisher pub_range( "/ultrasound", &range_msg);
char frameid[] = "/ultrasound";
   
void setup() 
{
  nh.initNode();
  nh.advertise(pub_range);
    
  range_msg.radiation_type = sensor_msgs::Range::ULTRASOUND;
  range_msg.header.frame_id =  frameid;
  range_msg.field_of_view = 0.1;  // fake
  range_msg.min_range = 0.0;
  range_msg.max_range = 60;
  
  pinMode(trigPin, OUTPUT);
  pinMode(echoPin, INPUT);
}
   
float getRange_Ultrasound()
{
    int val = 0;
    
    for(int i=0; i<4; i++) 
    {
     digitalWrite(trigPin, LOW);
     delayMicroseconds(2);
     digitalWrite(trigPin, HIGH);
     delayMicroseconds(10);
     digitalWrite(trigPin, LOW);
     
     duration = pulseIn(echoPin, HIGH);
     
     //Calculate the distance (in cm) based on the speed of sound.
     val += duration;
    }
    
    return val / 232.8 ;
}
   
long range_time;
   
void loop() 
{
   /* The following trigPin/echoPin cycle is used to determine the
    distance of the nearest object by bouncing soundwaves off of it. */
      
    if ( millis() >= range_time )
    {
       int r =0;
       range_msg.range = getRange_Ultrasound();
       range_msg.header.stamp = nh.now();
       pub_range.publish(&range_msg);
       range_time =  millis() + 50;
    }
     
    nh.spinOnce();
    delay(50); 
}


We can plot the distance value using the following command:


In [ ]:
roscore

In [ ]:
rosrun rosserial_python serial_node.py /dev/ttyACM*

In [ ]:
rqt_plot /ultrasound

<img src="img/img4.png" alt="Drawing" style="width: 700px;"/>

The center line indicates the current distance from the sensor. The upper line is the
max_range and line below is the minimum range.
